# Part 2: Postprocessing

In this section we'll explore postprocessing information in OpenMC's `StatePoint` files generated from OpenMC runs. These files contain a variety of information (tallies, timers, settings, etc.) from the simulation in a binary HDF5 format. The full set of information stored in these files can be found [here](https://docs.openmc.org/en/stable/io_formats/statepoint.html).

<span style="color:red">HINTS:</span>
* To run the commands, click on the relevant cell and hit the `shift + enter` keys simultaneously

Debugging tips
* Did you run all of the cells above your current cell? Try `Run -> Run All Above Selected Cell` from the menu
* Has the cell accidentally been switched from a `Code` cell to a `Raw` or `Markdown` cell? Check the dropdown menu that is to the far right of the save button
* If all else fails, switch over to the filled-in .ipynb file.

In [ ]:
import numpy as np
import openmc
import os
from pathlib import Path

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## Looking at simulation parameters

We'll begin by loading a statpoint file generated using the model from the previous section.

In [ ]:
!ls

In [ ]:
sp = openmc.StatePoint('statepoint.100.h5')

In [ ]:
sp.date_and_time

In [ ]:
print("{}, {}, {}".format(sp.n_batches, sp.n_inactive, sp.n_particles))

Note that this simulation uses more particles per batch than we did in Part1 to reduce the error on the tally results.

## Global values

The `StatePoint` also contains information about $k_{eff}$ for eigenvalue problems -- both the final value

In [ ]:
k_eff_part1 = sp.k_combined
print(k_eff_part1)

and the $k_{eff}$ for each generation

In [ ]:
plt.plot(sp.k_generation)

The k-eff plot indicates that this simulation ran fewer particles than would be ideal to get a well-converged solution, but for the purposes of the lesson it will do.

In [ ]:
sp.tallies

## Pandas Dataframes for tallies

In [ ]:
cell_tally = sp.tallies[1]
df = cell_tally.get_pandas_dataframe()

In [ ]:
df

In [ ]:
indices = df[('score')] == 'absorption'
sub_df = df[indices]
sub_df

In [ ]:
sub_df.plot(x='cell', y='mean', kind="bar", grid=True, ylabel="Absorption")

In [ ]:
# TODO Create a plot of the fission tally in each cell

## Plotting a Flux Tally

In [ ]:
flux_tally = sp.tallies[2]
flux_df = flux_tally.get_pandas_dataframe()

In [ ]:
flux_df.head(10)

In [ ]:
energy_filter = flux_tally.find_filter(openmc.EnergyFilter)
e_groups = energy_filter.values
e_groups

In [ ]:
indices = flux_df[('score')] == 'flux'
flux_vals = flux_df[indices]['mean'].values
# insert a zero to start for matplotlib
flux_vals = np.insert(flux_vals, 0, flux_vals[0])

In [ ]:
plt.loglog(e_groups, flux_vals, drawstyle='steps')
plt.xlabel('Energy [eV]')
plt.ylabel(r'Flux [$\frac{n-cm}{source\ particle}$]')

## Other ways to access tally data

We can also access the mean, standard deviation, and relative error directly on the `Tally` object.

In [ ]:
fission_mean = flux_tally.get_values(value='mean', scores=['fission'])
# insert a zero for matplotlib
fission_mean = np.insert(fission_mean, 0, fission_mean[0])

In [ ]:
plt.loglog(e_groups, fission_mean, drawstyle='steps')
plt.xlabel('Energy [eV]')
plt.ylabel(r'Fission [$\frac{reactions}{source\ particle}$]')

## Generating multi-group cross sections

In Part 1 we created a multigroup cross section library and asked it to add all necessary tallies to the OpenMC simulation. Here, we'll extract those cross sections from the statepoint file.

Many Monte Carlo particle transport codes, including OpenMC, use continuous-energy nuclear cross section data. However, most deterministic neutron transport codes use multi-group cross sections defined over discretized energy bins or energy groups. These cross sections are defined using a set of conserving reaction rates generated by the additional tallies we saw above.

At a minimum, the library must contain the absorption cross section
($\sigma_{a,g}$) and a scattering matrix. If the problem is an eigenvalue
problem then all fissionable materials must also contain either a fission
production matrix cross section ($\nu\sigma_{f,g\rightarrow g'}$), or both
the fission spectrum data ($\chi_{g'}$) and a fission production cross
section ($\nu\sigma_{f,g}$), or, .  The library must also contain the
fission cross section ($\sigma_{f,g}$) or the fission energy release cross
section ($\kappa\sigma_{f,g}$) if the associated tallies are required by
the model using the library.

After a scattering collision, the outgoing particle experiences a change in both
energy and angle. The probability of a particle resulting in a given outgoing
energy group ($g'$) given a certain incoming energy group ($g$) is provided by
the scattering matrix data.  The angular information can be expressed either via
Legendre expansion of the particle's change-in-angle ($\mu$), a tabular
representation of the probability distribution function of $\mu$, or a
histogram representation of the same PDF.

First we'll want to re-create the multigroup cross section (MGXS) library from Part1. To do this we'll use the information from the `summary.h5` file that was loaded as part of the `StatePoint` object. The summary file contains the geometry, material, and cross section specifications for the problem as HDF5 data. The summary is added to the `StatePoint` whenever it is present in the same directory and is a useful way to extract information about a problem if the original XML inputs are not present. 

In [ ]:
sp_geom = sp.summary.geometry
sp_mats = sp.summary.materials
sp_mats

Here, we'll use the summary to get the geometry and materials so we can re-create the MGXS library from Part 1.

In [ ]:
mgxs_lib = openmc.mgxs.Library(sp_geom)
mgxs_lib.domain_type = "material"
mgxs_lib.domains = sp_mats

In [ ]:
mgxs_lib.energy_groups = openmc.mgxs.EnergyGroups(openmc.mgxs.GROUP_STRUCTURES['CASMO-25'])
mgxs_lib.mgxs_types = ['total', 'absorption', 'nu-fission', 'fission', 'nu-transport',
                       'nu-scatter matrix', 'multiplicity matrix', 'chi']
mgxs_lib.build_library()

Now that we have build the MGXS library, we can load all the necessary information from the `StatePoint` object with a simple call.

In [ ]:
mgxs_lib.load_from_statepoint(sp)

This command uses the information from the MGXS library to locate the correct set of tallies in the `StatePoint` file, extract the relevant tally data for the material and reaction types we've specified, and add multigroup cross sections to the library.

From here, we can examine the generated cross sections.

In [ ]:
nu_fission = mgxs_lib.get_mgxs(sp_mats[0], 'nu-fission')

In [ ]:
nu_fission_df = nu_fission.get_pandas_dataframe()

In [ ]:
nu_fission_df.head(10)

## Applying multi-group cross sections to the model

OpenMC can also perform simulations with a discretized energy space using the cross sections we've generated above.

First, we'll create a new directory to perform this run, so we don't write over the existing files in case we want to redo this exercise later.

In [ ]:
Path('./mg_run').mkdir(exist_ok=True)
os.chdir('./mg_run')

Recall that when we created the MGXS Library, we provided both the original geometry and materials from the summary file. The `create_mg_mode` method of the library will generate new materials and geometry for the model that apply the multi-group cross sections we've generated here.

In [ ]:
mgxs_file, mg_materials, mg_geometry = mgxs_lib.create_mg_mode(xsdata_names=['uo2', 'zirc', 'water'])

In [ ]:
mg_materials

We can then apply these cross sections to our existing materials by setting the `cross_sections` parameter of the `materials` object. The multi-group cross sections will also need to be exported to OpenMC's HDF5 format.

In [ ]:
mgxs_file.export_to_hdf5()  # default filename is 'mgxs.h5'

In [ ]:
!ls

In [ ]:
mg_materials.cross_sections = './mgxs.h5'

In [ ]:
settings = openmc.Settings()
point = openmc.stats.Point((0, 0, 0))
src = openmc.Source(space=point)
settings.source = src
settings.batches = 150
settings.inactive = 10
settings.particles = 1000

In [ ]:
# set mode to mult-group
settings.energy_mode = 'multi-group'
settings.export_to_xml()

In [ ]:
!ls

In [ ]:
openmc.run()

In [ ]:
#TODO: Generate the missing input files needed to run OpenMC

In [ ]:
openmc.run()


## <span style="color:red">Best practices when working with StatePoint files </span>


For the purposes of the workshop we've been working with a `StatePoint` object throughout the notebook. This has allowed us to interactively examine data stored in the file, but generally it's best to gather what information is needed from the `StatePoint` inside a context-manager like so:

```python
with openmc.StatePoint('statepoint.100.h5') as sp:
    keff = sp.k_combined
    tallies = sp.tallies
```

This ensures that the file handles of the HDF5 `statepoint.100.h5` and `summary.h5` files are released and another OpenMC simulation that overwrites these files can be performed withtout error.

Alternatively, one can use a `StatePoint` object as we have here and delete the `StatePoint` object to free these file handles.

```python
del sp
```

In [ ]:
with openmc.StatePoint('statepoint.150.h5') as multigroup_sp:
    k_eff_part2 = multigroup_sp.k_combined

In [ ]:
print("Continuous Energy k-eff: {}".format(k_eff_part1))
print("Multigroup k-eff: {}".format(k_eff_part2))
k_eff_diff = k_eff_part1 - k_eff_part2
print("Difference: {}".format(k_eff_diff))